## Preprocessing version 4
### Details
this version 
- CNN/DM
- uses all data
- has validation split
- uses pretrained embedding

In [1]:
doc_name = "data2/train.txt.src"
summ_name = "data2/train.txt.tgt.tagged"
vocab_name = "vocab.json"
data_seq_name = "train_seq.json"
valid_seq_name = "valid_seq.json"
# min_count = 7
vocab_size = 90000
num_threads = 4
# n_dim = 300
corpus_size = 287227
validation_split = 0.01
epochs = 0

In [2]:
import os
import csv
import json
import numpy as np
from nltk import word_tokenize as tokenize
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool
from gensim.models.callbacks import CallbackAny2Vec

In [3]:
def getlines(name):
    total = !wc -l {name}
    return int(total[0].split()[0])

In [ ]:
summaries = []
articles = []
with open(doc_name, newline='', encoding='utf-8') as f:
    total = getlines(doc_name)
    for i,line in tqdm(enumerate(f), total=total):        
        line = line.strip()
        articles.append(line)        
with open(summ_name, newline='', encoding='utf-8') as f:
    total = getlines(summ_name)
    for i,line in tqdm(enumerate(f), total=total):        
        line = line.strip()
        line = line.replace("<t>", "").replace("</t>", "")
        summaries.append(line)

In [ ]:
summaries = summaries[:min(corpus_size, len(summaries))]
articles = articles[:min(corpus_size, len(summaries))]

In [ ]:
def task(s):
    words = tokenize(s)
    return ['<bos>'] + words + ['<eos>']

with Pool(num_threads) as p:
    chunksize = 200
    summaries = list(tqdm(p.imap(task, summaries, chunksize=chunksize), total=len(summaries)))
    chunksize = 50
    articles = list(tqdm(p.imap(task, articles, chunksize=chunksize), total=len(articles)))

In [ ]:
num_summ = len(summaries)
index = 99999
print("[summary]", summaries[index])
print("[documen]", articles[index])

In [ ]:
from gensim.models import FastText
model = FastText(size=n_dim, window=5, min_count=1, workers=num_threads)
model.build_vocab(summaries+articles)
# total_examples = model.corpus_count
# print(total_examples)
# logger = EpochLogger()
# model.train(data, total_examples=total_examples, epochs=epochs, callbacks=[logger])

In [ ]:
VOCAB_SZ = len(model.wv.vocab)
print(VOCAB_SZ)

In [ ]:
print(model.wv.vocab['isabelle'])

In [ ]:
top_vocab = sorted(model.wv.vocab.items(), key=lambda a: -a[1].count)[:(vocab_size-2)]
vocab = {w:i for i, (w,voc) in enumerate(top_vocab)}
vocab['<unk>'] = len(vocab)
vocab['<pad>'] = len(vocab)
print(len(vocab))

In [ ]:
json.dump(vocab, open(vocab_name, "w"))

In [ ]:
seqdata = {'summary':[], 'document':[]}
valseqdata = {'summary':[], 'document':[]}
# num_summ = len(data) / 2
val_set = np.random.randint(0, num_summ, size=int(validation_split*num_summ))
for i,words in tqdm(enumerate(summaries), total=num_summ):    
    seq = []
    for w in words:        
        try:
            wid = vocab[w]
        except KeyError:
            wid = vocab["<unk>"]
#             wid = OOVindex(w)
        seq.append(wid)
    if i in val_set:
        valseqdata['summary'].append(seq)
    else:
        seqdata['summary'].append(seq)
for i,words in tqdm(enumerate(articles), total=num_summ):
    seq = []
    for w in words:
        try:
            wid = vocab[w]
        except KeyError:
            wid = vocab["<unk>"]
#             wid = OOVindex(w)
        seq.append(wid)
    if i in val_set:
        valseqdata['document'].append(seq)
    else:
        seqdata['document'].append(seq)
print(len(seqdata['document']), len(valseqdata['document']))

In [ ]:
json.dump(seqdata, open(data_seq_name, "w"))
json.dump(valseqdata, open(valid_seq_name, "w"))

In [ ]:
lengths = []
key = 'summary'
for doc in seqdata[key]:
    lens = len(doc)
    lengths.append(lens)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(lengths, 50)
plt.title(key +' length')
plt.show()

In [ ]:
take a break
# 2000 -> 100

### Word Embedding

In [ ]:
import numpy as np
import json
from gensim.models import FastText
from tqdm import tqdm_notebook as tqdm
pretrained = 'data2/cc.en.300.bin'
vocab_name = "preprocessing-cnn/vocab.json"
matrixname = "wv_matrix"
pmodel = FastText.load_fasttext_format(pretrained) 
vocab = json.load(open(vocab_name, "r"))
n_dim = 300

In [ ]:
def l2_normd_np(a):
    b = np.linalg.norm(a)
    return 0*a if b == 0 else a / b
VOCAB_SZ = len(vocab)

oovs = []
wv_matrix = (np.random.rand(VOCAB_SZ, n_dim) - 0.5) / 5.0
for word, wid in tqdm(vocab.items()):    
    try:
        vec = pmodel.wv[word]
    except KeyError:
        oovs.append(word)
        vec = (np.random.rand(n_dim) - 0.5) / 5.0
    wv_matrix[wid] = l2_normd_np(vec)

print(len(wv_matrix))
print("done.")

In [ ]:
print(len(oovs))
print(oovs)

In [ ]:
np.save(matrixname, wv_matrix)